In [2]:
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import os

In [101]:
def getAgentFiles():   
    folders = next(os.walk('./agent_icons'))[1]
    folders.pop()
    folders.pop(0)
    
    allFileNames = []
    for folder in folders:
        agentIconsFileList = os.listdir(f"./agent_icons/{folder}")
        for x in range(len(agentIconsFileList)):
            allFileNames.append(f"agent_icons/{folder}/" + agentIconsFileList[x])
    return allFileNames

In [139]:
print(getAgentFiles())

['agent_icons/cypher/1.png', 'agent_icons/cypher/2.png', 'agent_icons/cypher/3.png', 'agent_icons/cypher/4.png', 'agent_icons/sage/1.png', 'agent_icons/sage/2.png', 'agent_icons/sage/3.png', 'agent_icons/sage/4.png', 'agent_icons/sova/1.png', 'agent_icons/sova/2.png', 'agent_icons/sova/3.png', 'agent_icons/sova/4.png']


In [211]:
def trackAgent(img_gray, img_rgb, agent_name, threshold, minimap):
    subImage = cv2.imread(agent_name, 0)
    w, h = subImage.shape[::-1]
    res = cv2.matchTemplate(img_gray, subImage,cv2.TM_CCOEFF_NORMED)
    loc = np.where( res >= threshold)
    for pt in zip(*loc[::-1]):
        img_rgb = cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
        SCALE_FACTOR =3.7275
        SHIFT_FACTOR = -103
        newX = (int ((pt[0] + 0.5 * w) * SCALE_FACTOR)) + SHIFT_FACTOR
        newY = (int ((pt[1] + 0.5 * h) * SCALE_FACTOR)) + SHIFT_FACTOR
        minimap = cv2.circle(minimap, (newX, newY) , radius=5, color=(0, 0, 255), thickness=-1)
    return ({"radar":img_rgb, "map":minimap})



In [217]:
def trackAllAgents(img_gray, img_rgb, threshold):
    minimap = cv2.imread('./maps/Ascent_Minimap.png', cv2.IMREAD_COLOR)
    for agent in getAgentFiles():
        newImages = trackAgent(img_gray, img_rgb, agent, threshold, minimap)
        img_rgb = newImages['radar']
        minimap = newImages['map']
    
    cv2.imshow('radar',img_rgb)
    minimap = cv2.resize(minimap, (0,0), fx=0.5, fy=0.5)
    cv2.imshow('map', minimap)
    cv2.waitKey(1)

In [227]:
#all icons, video
cap = cv2.VideoCapture("game.mp4")

cap.set(3, 1280)
cap.set(4, 720)
cap.set(1, 0)

t_end = time.time() + 5
while time.time() < t_end:
    ret, frame = cap.read()
    frame = frame[0:300, 0:300] #300 for og vid
    img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    trackAllAgents(img_gray, frame, 0.95)
cv2.destroyAllWindows()

In [122]:
#specific icon, frame


frame = cv2.imread('cropped.png', cv2.IMREAD_COLOR)
frame = frame[0:500, 0:500] #300 for og vid
img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
cv2.imshow('res', trackAgent(img_gray, frame, 'agent_icons/cypher.png', 0.95))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [59]:
#specific icon, video

cap = cv2.VideoCapture("game.mp4")
cap.set(1, 0)
t_end = time.time() + 5
while time.time() < t_end:
    ret, frame = cap.read()
    frame = frame[0:500, 0:500] #300 for og vid
    img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('res', trackAgent(img_gray, frame, 'agent_icons/cypher.png', 0.9))
    cv2.waitKey(1)
cv2.destroyAllWindows()